In [1]:
!pip install transformers accelerate datasets ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 153.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 125.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!huggingface-cli login --token ''

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
from datasets import load_dataset

from datasets import DatasetDict, Dataset

# Set random seed for reproducibility
torch.random.manual_seed(0)

# Load the model and tokenizer
# change model her and just run the code 
model = AutoModelForCausalLM.from_pretrained(
    "wasifis/cited_phi3_mini_instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

# Initialize the pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

# Load the dataset
dataset_dict = load_dataset("wasifis/rag-test-gt")

# Convert to pandas DataFrame
df = pd.DataFrame(dataset_dict["default"])

# Prepare the base context
base_context = [
    {"role": "system", "content": "You are a helpful AI assistant."}
]

# Function to generate response for each question with context
def generate_response(question, context):
    messages = base_context + [{"role": "user", "content": context}, {"role": "user", "content": question}]
    generation_args = {
        "max_new_tokens": 500,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False,
    }
    output = pipe(messages, **generation_args)
    return output[0]['generated_text']

# Apply the function to each question and context in the DataFrame
df['answer'] = df.apply(lambda row: generate_response(row['question'], row['contexts']), axis=1)

# Save the updated DataFrame to a new CSV file
df.to_csv('generated_testset_with_answers.csv', index=False)

config.json:   0%|          | 0.00/3.43k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


...

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Generating default split:   0%|          | 0/20 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


In [ ]:
import pandas as pd
from datasets import Dataset
import os
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = ""

# Load the CSV file into a DataFrame
csv_file_path = 'generated_testset_with_answers.csv'
df = pd.read_csv(csv_file_path)

# Inspect the DataFrame to understand its structure
print(df.dtypes)

# Ensure that the 'contexts' column is of type list of strings
if 'contexts' in df.columns:
    df['contexts'] = df['contexts'].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else [x])

# Ensure that the 'ground_truths' column is of type list of strings
if 'ground_truths' in df.columns:
    df['ground_truths'] = df['ground_truths'].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else [x])

# Convert the DataFrame to a dictionary
data_samples = df.to_dict(orient='list')

# Create a Dataset from the dictionary
dataset = Dataset.from_dict(data_samples)

# Define the metrics
metrics = [faithfulness, answer_correctness]

# Evaluate the dataset using the specified metrics
score = evaluate(dataset, metrics=metrics)

# Convert the score to a pandas DataFrame and print it
print(score.to_pandas())
